In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pysindy as ps 
from scipy.integrate import solve_ivp
from utils import *
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

We have relaxed the hard constraint slightly and solved the trapping SINDy objective function with inequality constraints dictating the amount of deviation allowed from perfect skew-symmetry in the quadratic coefficients:
$$ argmin_{\mathbf{\xi},\mathbf m}\|\dot{\mathbf a} - \mathbf \Theta(\mathbf a) \mathbf{\xi}\|^2 + \gamma R(\mathbf \xi)  + \eta^{-1} \lambda_1(\mathbf A)  \quad s.t. \quad -\epsilon_Q \leq Q_{ijk} + Q_{jik} + Q_{kji} \leq \epsilon_Q.$$ 
This allowed us to build locally Lyapunov stable models, and adjust the size of the local stability radius by varying $\epsilon_Q$. However, two other loss terms that can be used as alternatives to increase the size of the stability radius while avoiding extra constraints:
$$\alpha^{-1}\|Q_{ijk}\|$$
and
$$\beta^{-1}\|Q_{ijk} + Q_{jki} + Q_{kij}\|.$$
We can combine all of these options into the following unconstrained optimization problem:
$$argmin_{\mathbf{\xi},\mathbf m}\|\dot{\mathbf a} - \mathbf \Theta(\mathbf a) \mathbf{\xi}\|^2 + \gamma R(\mathbf \xi)  + \eta^{-1} \lambda_1(\mathbf A) + \alpha^{-1}\|Q_{ijk}\| + \beta^{-1}\|Q_{ijk} + Q_{jki} + Q_{kij}\|.$$
We now solve this problem for $\alpha \gg \beta$, $\alpha \ll \beta$, and $\alpha \sim \beta \sim 1.$

A conservative estimate of the local stability is:
$$\rho_+ = \frac{3}{2r^{\frac{3}{2}}\epsilon_Q} \left( \sqrt{\lambda^2_{\text{max}}(\textbf{A}_S) - \frac{4r^{\frac{3}{2}}\epsilon_Q}{3}\|\mathbf{d}\|_2} - \lambda_{\text{max}}(\textbf{A}_S) \right).$$
And the radius of the trapping region is given by:
$$\rho_- = -\frac{3}{2r^{\frac{3}{2}}\epsilon_Q} \left( \sqrt{\lambda^2_{\text{max}}(\textbf{A}_S) - \frac{4r^{\frac{3}{2}}\epsilon_Q}{3}\|\mathbf{d}\|_2} + \lambda_{\text{max}}(\textbf{A}_S) \right).$$

### Dysts database contains a number of quadratically nonlinear chaotic systems with the special energy-preserving nonlinear symmetry.
You will need to install the dysts database with 'pip install dysts' or similar command (see https://github.com/williamgilpin/dysts)

In [2]:
import dysts.flows as flows

# Initialize quadratic SINDy library, with custom ordering 
# to be consistent with the constraint
library_functions = [lambda x:x, lambda x, y:x * y, lambda x:x ** 2]
library_function_names = [lambda x:x, lambda x, y:x + y, lambda x:x + x]
sindy_library = ps.CustomLibrary(library_functions=library_functions, 
                                 function_names=library_function_names,
                                 include_bias=True)

# Initialize integrator keywords for solve_ivp to replicate the odeint defaults
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-15
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-10

# Arneodo does not have the Lyapunov spectrum calculated so omit it.
# HindmarshRose and AtmosphericRegime seem to be poorly sampled
# by the dt and dominant time scales used in the database, so we omit them.
trapping_system_list = np.array([2, 3, 7, 10, 18, 24, 27, 29, 30, 34, 40, 46, 47, 66, 67])
systems_list = [
                "Aizawa", "Bouali2", 
                "GenesioTesi", "HyperBao", "HyperCai", "HyperJha", 
                "HyperLorenz", "HyperLu", "HyperPang", "Laser",
                "Lorenz", "LorenzBounded", "MooreSpiegel", "Rossler", "ShimizuMorioka",
                "HenonHeiles", "GuckenheimerHolmes", "Halvorsen", "KawczynskiStrizhak",
                "VallisElNino", "RabinovichFabrikant", "NoseHoover", "Dadras", "RikitakeDynamo",
                "NuclearQuadrupole", "PehlivanWei", "SprottTorus", "SprottJerk", "SprottA", "SprottB",
                "SprottC", "SprottD", "SprottE", "SprottF", "SprottG", "SprottH", "SprottI", "SprottJ",
                "SprottK", "SprottL", "SprottM", "SprottN", "SprottO", "SprottP", "SprottQ", "SprottR",
                "SprottS", "Rucklidge", "Sakarya", "RayleighBenard", "Finance", "LuChenCheng",
                "LuChen", "QiChen", "ZhouChen", "BurkeShaw", "Chen", "ChenLee", "WangSun", "DequanLi",
                "NewtonLiepnik", "HyperRossler", "HyperQi", "Qi", "LorenzStenflo", "HyperYangChen", 
                "HyperYan", "HyperXu", "HyperWang", "Hadley",
               ]
alphabetical_sort = np.argsort(systems_list)
systems_list = (np.array(systems_list)[alphabetical_sort])[trapping_system_list]

# attributes list
attributes = [
    "maximum_lyapunov_estimated",
    "lyapunov_spectrum_estimated",
    "embedding_dimension",
    "parameters",
    "dt",
    "hamiltonian",
    "period",
    "unbounded_indices"
]

# Get attributes
all_properties = dict()
for i, equation_name in enumerate(systems_list):
    eq = getattr(flows, equation_name)()
    attr_vals = [getattr(eq, item, None) for item in attributes]
    all_properties[equation_name] = dict(zip(attributes, attr_vals))
    
# Get training and testing trajectories for all the experimental systems 
n = 1000  # Trajectories with 1000 points
pts_per_period = 100  # sample them with 100 points per period
n_trajectories = 1  # generate 5 trajectories starting from different initial conditions on the attractor
all_sols_train, all_t_train, all_sols_test, all_t_test = load_data(
    systems_list, all_properties, 
    n=n, pts_per_period=pts_per_period,
    random_bump=False,  # optionally start with initial conditions pushed slightly off the attractor
    include_transients=False,  # optionally do high-resolution sampling at rate proportional to the dt parameter 
    n_trajectories=n_trajectories
)

0 BurkeShaw(name='BurkeShaw', params={'e': 13, 'n': 10}, random_state=None)
1 Chen(name='Chen', params={'a': 35, 'b': 3, 'c': 28}, random_state=None)
2 Finance(name='Finance', params={'a': 0.001, 'b': 0.2, 'c': 1.1}, random_state=None)
3 Hadley(name='Hadley', params={'a': 0.2, 'b': 4, 'f': 9, 'g': 1}, random_state=None)
4 HyperPang(name='HyperPang', params={'a': 36, 'b': 3, 'c': 20, 'd': 2}, random_state=None)
5 HyperYangChen(name='HyperYangChen', params={'a': 30, 'b': 3, 'c': 35, 'd': 8}, random_state=None)
6 Lorenz(name='Lorenz', params={'beta': 2.667, 'rho': 28, 'sigma': 10}, random_state=None)
7 LorenzStenflo(name='LorenzStenflo', params={'a': 2, 'b': 0.7, 'c': 26, 'd': 1.5}, random_state=None)
8 LuChen(name='LuChen', params={'a': 36, 'b': 3, 'c': 18}, random_state=None)
9 NoseHoover(name='NoseHoover', params={'a': 1.5}, random_state=None)
10 RayleighBenard(name='RayleighBenard', params={'a': 30, 'b': 5, 'r': 18}, random_state=None)
11 SprottA(name='SprottA', params={}, random_stat

In [3]:
num_attractors = len(systems_list)

# Calculate some dynamical properties
lyap_list = []
dimension_list = []
param_list = []

# Calculate various definitions of scale separation
scale_list_avg = []
scale_list = []
linear_scale_list = []

for system in systems_list:
    lyap_list.append(all_properties[system]['maximum_lyapunov_estimated'])
    dimension_list.append(all_properties[system]['embedding_dimension'])
    param_list.append(all_properties[system]['parameters'])
    
    # Ratio of dominant (average) to smallest timescales
    scale_list_avg.append(all_properties[system]['period'] / all_properties[system]['dt'])


# Get the true coefficients for each system
true_coefficients = make_dysts_true_coefficients(systems_list, 
                                                 all_sols_train, 
                                                 dimension_list, 
                                                 param_list)

In [4]:
for i in range(len(systems_list)):
# define parameters
    print(i, systems_list[i])
    r = dimension_list[i]
    N = int((r ** 2 + 3 * r) / 2.0)
    # make training and testing data
    t = all_t_train[systems_list[i]][0]
    x_train = all_sols_train[systems_list[i]][0]
    x_test = all_sols_test[systems_list[i]][0]

    # define hyperparameters
    threshold = 0
    max_iter = 2000
    eta = 1.0e3
    constraint_zeros, constraint_matrix = make_constraints(r)

    alpha_m = 4e-2 * eta  # default is 1e-2 * eta so this speeds up the code here
    accel = True  # use acceleration for the update of (m, A), sometimes is faster

    # run trapping SINDy
    sindy_opt = ps.TrappingSR3(threshold=threshold, eta=eta, alpha_m=alpha_m,
                               accel=accel, max_iter=max_iter, gamma=-1,
                               constraint_lhs=constraint_matrix,
                               constraint_rhs=constraint_zeros,
                               constraint_order="feature",
                               verbose=True
                               )
    model = ps.SINDy(
        optimizer=sindy_opt,
        feature_library=sindy_library,
    )
    model.fit(x_train, t=t, quiet=True)
    model.print()

    Xi = model.coefficients().T
    Q = np.tensordot(sindy_opt.PQ_, Xi, axes=([4, 3], [0, 1]))
    print('nonlinearity preservation breaking = ', np.max(np.abs((Q + np.transpose(Q, [1, 2, 0]) + np.transpose(Q, [2, 0, 1])))))
    xdot_test = model.differentiate(x_test, t=t)
    xdot_test_pred = model.predict(x_test)
    x_train_pred = model.simulate(x_train[0, :], t, integrator_kws=integrator_keywords)
    x_test_pred = model.simulate(x_test[0, :], t, integrator_kws=integrator_keywords)
    plt.figure(i + 1)
    plt.plot(x_train[:, 0], x_train[:, 1])
    plt.plot(x_train_pred[:, 0], x_train_pred[:, 1])
    check_stability(r, Xi, np.eye(r), sindy_opt, 1.0)
    
    Xi_true = true_coefficients[i][:, :N + 1].T
    boundvals = np.zeros((r, 2))
    boundmax = 1000
    boundmin = -1000
    boundvals[:, 0] = boundmin
    boundvals[:, 1] = boundmax

    PL_tensor = sindy_opt.PL_
    PM_tensor = sindy_opt.PM_
    L = np.tensordot(PL_tensor, Xi_true, axes=([3, 2], [0, 1]))
    Q = np.tensordot(PM_tensor, Xi_true, axes=([4, 3], [0, 1]))
    
    # run simulated annealing on the true system to make sure the system is amenable to trapping theorem
    algo_sol = anneal_algo(obj_function, bounds=boundvals, 
                           args=(L, Q, np.eye(r)), 
                           maxiter=500)
    opt_m = algo_sol.x
    opt_energy = algo_sol.fun
    print('Simulated annealing managed to reduce the largest eigenvalue of A^S to eig1 = ', 
          opt_energy, '\n')
    print('Optimal m = ', opt_m)
#     make_progress_plots(r, sindy_opt)
plt.show()

0 BurkeShaw
 Iter ... |y-Xw|^2 ... |Pw-A|^2/eta ... |w|_1 ... |Qijk|/a ... |Qijk+...|/b ...   Total:
    0 ... 4.924e+01 ... 1.004e-02 ... 0.00e+00 ... 4.90e-19 ... 1.56e-47 ... 4.93e+01
  200 ... 4.924e+01 ... 2.586e-03 ... 0.00e+00 ... 4.90e-19 ... 1.30e-47 ... 4.92e+01
  400 ... 4.924e+01 ... 2.586e-03 ... 0.00e+00 ... 4.90e-19 ... 1.65e-47 ... 4.92e+01
  600 ... 4.924e+01 ... 2.585e-03 ... 0.00e+00 ... 4.90e-19 ... 2.44e-47 ... 4.92e+01
  800 ... 4.924e+01 ... 2.585e-03 ... 0.00e+00 ... 4.90e-19 ... 1.96e-47 ... 4.92e+01
 1000 ... 4.924e+01 ... 2.585e-03 ... 0.00e+00 ... 4.90e-19 ... 5.95e-48 ... 4.92e+01
 1200 ... 4.924e+01 ... 2.584e-03 ... 0.00e+00 ... 4.90e-19 ... 4.88e-48 ... 4.92e+01
 1400 ... 4.924e+01 ... 2.584e-03 ... 0.00e+00 ... 4.90e-19 ... 1.06e-47 ... 4.92e+01
 1600 ... 4.924e+01 ... 2.584e-03 ... 0.00e+00 ... 4.90e-19 ... 2.44e-47 ... 4.92e+01
 1800 ... 4.924e+01 ... 2.583e-03 ... 0.00e+00 ... 4.90e-19 ... 3.52e-48 ... 4.92e+01
(x0)' = -0.001 1 + -9.869 x0 + -9.921 x

 Iter ... |y-Xw|^2 ... |Pw-A|^2/eta ... |w|_1 ... |Qijk|/a ... |Qijk+...|/b ...   Total:
    0 ... 8.716e+02 ... 2.591e-01 ... 0.00e+00 ... 4.83e-21 ... 4.46e-46 ... 8.72e+02
  200 ... 8.717e+02 ... 6.113e-02 ... 0.00e+00 ... 4.82e-21 ... 3.30e-46 ... 8.72e+02
  400 ... 8.721e+02 ... 2.184e-02 ... 0.00e+00 ... 4.82e-21 ... 1.12e-45 ... 8.72e+02
  600 ... 8.727e+02 ... 1.071e-02 ... 0.00e+00 ... 4.81e-21 ... 1.87e-45 ... 8.73e+02
  800 ... 8.731e+02 ... 6.538e-03 ... 0.00e+00 ... 4.81e-21 ... 7.55e-46 ... 8.73e+02
 1000 ... 8.735e+02 ... 4.632e-03 ... 0.00e+00 ... 4.80e-21 ... 9.74e-46 ... 8.74e+02
 1200 ... 8.739e+02 ... 3.630e-03 ... 0.00e+00 ... 4.80e-21 ... 4.69e-45 ... 8.74e+02
 1400 ... 8.741e+02 ... 3.049e-03 ... 0.00e+00 ... 4.80e-21 ... 1.17e-45 ... 8.74e+02
 1600 ... 8.744e+02 ... 2.687e-03 ... 0.00e+00 ... 4.80e-21 ... 3.81e-46 ... 8.74e+02
 1800 ... 8.746e+02 ... 2.450e-03 ... 0.00e+00 ... 4.80e-21 ... 3.27e-46 ... 8.75e+02
(x0)' = 1.154 1 + -29.111 x0 + 29.385 x1 + -0.082 x

capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.

KeyboardInterrupt



Error in callback <function _draw_all_if_interactive at 0x10ef44f70> (for post_execute):



KeyboardInterrupt



Error in callback <function flush_figures at 0x124333940> (for post_execute):



KeyboardInterrupt

